In [1]:
from pathlib import Path
import random as rd
import numpy as np
import pandas as pd
import operator
import copy
import sklearn
from sklearn import cluster
from sklearn import metrics
#import tensorflow as tf
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior() 

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

import imblearn.over_sampling as smote

import matplotlib.pyplot as plt
import csv
from matplotlib.axis import Axis   
import matplotlib.ticker as ticker 
import time
from glob import glob

import collections
import re
import argparse
import csv
import os
import sys

In [2]:
def get_test_coverage(coverage_file, spectra_file):
    
    #spectrum data and their labels. Main data for the project.
    spectrum_data = list() 
    spectrum_labels = list()
    
    """ if(Path(spectra_file)).exists():
        with open(spectra_file) as name_file:
            element_names = {i: name.strip() for i, name in enumerate(name_file)}

        n_elements = len(element_names) """
    
    if Path(coverage_file).exists():
        matrix_file = open(coverage_file)
        for line in matrix_file.readlines():

            line = line.split(" ")
            if line[-1]=='+\n':
                line[-1] = 0 #0 for passing test
            else: # always '-'
                line[-1] = 1 #1 for failing test
                            
            #y = line[-1]
            #X = line[:-1]

            y = float(line[-1])  # Convert y to float
            X = [float(x) for x in line[:-1]]  # Convert X to float
            
            """  if y==1:
                for i in range(len(X)): #Filter down to union of components executed by faulty test cases.
                    if X[i]=="1":
                        important_column_numbers.add(i)  """
                        
            spectrum_data.append(X)
            spectrum_labels.append(y)

    """  spectrum_data_reduced = list()
    important_column_numbers = list(important_column_numbers)
    
    for line in spectrum_data:
        line = [int(i) for i in line]
        spectrum_data_reduced.append([line[i] for i in important_column_numbers]) #NhamCT-chi loc nhung line co thuc hien TC. CHange to numpy
    spectrum_data_reduced= np.array(spectrum_data_reduced) """
   
    return spectrum_data, spectrum_labels#, important_column_numbers

In [3]:
def create_model(input_shape):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_shape, 1)),  
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output là 1 nếu fail, 0 nếu pass
])

    model.compile(loss='binary_crossentropy',
        optimizer=RMSprop(learning_rate=1e-4),
        metrics=['accuracy'])
  
    return model

In [6]:
""" PROJECTS = ['Closure', 'Lang', 'Chart', 'Math', 'Mockito', 'Time']
PROJECT_BUGS = [
    [str(x) for x in range(1, 134)],
    [str(x) for x in range(1, 66)],
    [str(x) for x in range(1, 27)],
    [str(x) for x in range(1, 107)],
    [str(x) for x in range(1, 39)],
    [str(x) for x in range(1, 28)]
] """

PROJECTS = ['Chart']
PROJECT_BUGS = [[str(x) for x in range(1, 9)]]

input_data_dir="D:\TEMP\Matrix"
output_dir = "D:\TEMP\Sus"
fig_dir = "D:\TEMP\Fig"
model_dir = "Models"

EPOCHS = 10

In [ ]:
coverage_files = sorted(glob(os.path.join(input_data_dir, '*coverage')))
spectra_files = sorted(glob(os.path.join(input_data_dir, '*spectra')))
accuracies = []
for project, bugs in zip(PROJECTS, PROJECT_BUGS):
    for bug in bugs:
        coverage_file = os.path.join(input_data_dir,'%s-%s-coverage' %(project, bug))
        spectra_file = os.path.join(input_data_dir,'%s-%s-spectra' %(project, bug))

        spectrum_data, spectrum_labels = get_test_coverage(coverage_file, spectra_file)
        spectrum_data = tf.convert_to_tensor(spectrum_data, dtype=None,dtype_hint=None, name=None)
        spectrum_labels = tf.convert_to_tensor(spectrum_labels,dtype=None,dtype_hint=None, name=None)

        with open(spectra_file) as name_file:
            element_names = {i: name.strip() for i, name in enumerate(name_file)}
        n_elements = len(element_names)
        
        #imbalance process
        sm = smote.RandomOverSampler(random_state=42)
        X, y  = sm.fit_resample(spectrum_data, spectrum_labels)

        tf.keras.backend.clear_session()

        model = create_model(n_elements)
        history = model.fit(X, y, epochs=EPOCHS, batch_size=1, shuffle=True, validation_split=0.2)
        model.save(save_format='h5',filepath=os.path.join(model_dir,'%s-%s-cnn.h5'%(project, bug)))

        accuracies.append((project, bug, history.history['accuracy'], history.history['val_accuracy']))

        #Plot accuracy
        train_accuracy = history.history['loss']
        val_accuracy = history.history['val_loss']

        plt.plot(train_accuracy, label='Training Loss')
        plt.plot(val_accuracy, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('%s-%s' %(project, bug))
        #plt.show()
        plt.savefig(os.path.join(fig_dir,'%s-%s-cnn.jpg' % (project, bug)))
        plt.close('all')

        suspiciousness = []
        virtual_test_set = [[1 if j==i else 0 for j in range(n_elements)] for i in range(n_elements)]
        for i in range(n_elements):
            virtual_feature = np.array(virtual_test_set[i]).reshape(1,-1)
            output = model.predict(virtual_feature)
            suspiciousness.append(output[0])
        
        with open(os.path.join(output_dir, '%s-%s-cnn-suspiciousness' % (project, bug)), 'w') as output_file:
            writer = csv.DictWriter(output_file, ['Statement','Suspiciousness'])
            writer.writeheader()
            for element in range(n_elements):
                writer.writerow({ 'Statement': element_names[element],'Suspiciousness': suspiciousness[element]})

with open(os.path.join(output_dir, '%s-%s-cnn-accuracy' % (project, bug)), 'w', newline='') as accuary_file:
    writer_acc = csv.writer(accuary_file)
    for row in accuracies:
        writer_acc.writerow(row)
